In [1]:
import json
from itertools import islice
import pandas as pd
import pickle
import re
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torch.utils.data
import math
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


### Train 데이터 가공

#### 데이터 불러오기

In [ ]:
def open_json(url):
  with open(url) as f:
    data = json.load(f)
  return data

In [ ]:
# 초등
data1 = open_json('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/DATA/01-1.정식개방데이터/Training/01.원천데이터/TS_01. 학교급_01. 초등/상담기록_데이터_초등학교.json')
# 중등
data2 = open_json('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/DATA/01-1.정식개방데이터/Training/01.원천데이터/TS_01. 학교급_02. 중등/상담기록_데이터_중학교.json')
# 고등
data3 = open_json('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/DATA/01-1.정식개방데이터/Training/01.원천데이터/TS_01. 학교급_03. 고등/상담기록_데이터_고등학교.json')

#### [실행x] 데이터 구조 파악

In [ ]:
trying = dict(islice(data1.items(), 2))

In [ ]:
trying

{'0': {'meta': {'student_idx': 'S-0155',
   'counseling_idx': 1,
   'counsellor_idx': 'T-0015',
   'counselling_purpose': '학교 진로 상담 참여',
   'counselling_satisfaction': 5,
   'counselling_date': '2022-09-22'},
  'conversation': [{'conv_idx': 1,
    'conv_category': '기타',
    'self_eval': [3],
    'utterances': [{'speaker_idx': 'S-0155',
      'utterance': '안녕하세요~',
      'utterance_idx': 1,
      'utterance_delaytime': 0.0},
     {'speaker_idx': 'T-0015',
      'utterance': '어서 오세요 [이모티콘]',
      'utterance_idx': 2,
      'utterance_delaytime': 16.7},
     {'speaker_idx': 'T-0015',
      'utterance': '방학인데 휴가는 다녀왔어요?',
      'utterance_idx': 3,
      'utterance_delaytime': 91.48},
     {'speaker_idx': 'S-0155',
      'utterance': '네!',
      'utterance_idx': 4,
      'utterance_delaytime': 22.13},
     {'speaker_idx': 'T-0015',
      'utterance': '어디로 다녀왔나요?',
      'utterance_idx': 5,
      'utterance_delaytime': 1.35},
     {'speaker_idx': 'S-0155',
      'utterance': '충주요',
      'ut

In [ ]:
trying['0'] # 첫 번째 학생의 대화

# 첫 번째 학생이랑 한 대화들을 가져온 건데..
# conv_idx로 나눠져 있어
# 이 상태면 list
first = trying['0']['conversation']

len(first) # 5
# conv_idx 개수만큼 있구나

first[0] # 이게 첫 대화에서 파트 1
type(first[0]) # 이건 dict

first_talk = first[0]['utterances']

conv = []
for i in range(len(first_talk)):

  conv.append(first_talk[i]['utterance'])

In [ ]:
conversation = []
for i in range(len(first_talk)):
  conversation.append({
      'speaker' : first_talk[i]['speaker_idx'],
      'utterance' : first_talk[i]['utterance']
  })
conversation

[{'speaker': 'S-0155', 'utterance': '안녕하세요~'},
 {'speaker': 'T-0015', 'utterance': '어서 오세요 [이모티콘]'},
 {'speaker': 'T-0015', 'utterance': '방학인데 휴가는 다녀왔어요?'},
 {'speaker': 'S-0155', 'utterance': '네!'},
 {'speaker': 'T-0015', 'utterance': '어디로 다녀왔나요?'},
 {'speaker': 'S-0155', 'utterance': '충주요'},
 {'speaker': 'T-0015', 'utterance': '네~ 조부모님 댁인가요?'},
 {'speaker': 'S-0155', 'utterance': '여행이요 하하'}]

In [ ]:
conv

['안녕하세요~',
 '어서 오세요 [이모티콘]',
 '방학인데 휴가는 다녀왔어요?',
 '네!',
 '어디로 다녀왔나요?',
 '충주요',
 '네~ 조부모님 댁인가요?',
 '여행이요 하하']

In [ ]:
print("총 학생 수:", len(data1)) # 2376 명의 학생이 존재
print("마지막 학생과의 대화: /n", data1['2375'])

총 학생 수: 2376
마지막 학생과의 대화: /n {'meta': {'student_idx': 'S-6905', 'counseling_idx': 2, 'counsellor_idx': 'T-0026', 'counselling_purpose': '학교 진로 상담 참여', 'counselling_satisfaction': 3, 'counselling_date': '2022-08-07'}, 'conversation': [{'conv_idx': 1, 'conv_category': '직업이해', 'self_eval': [3], 'utterances': [{'speaker_idx': 'S-6905', 'utterance': '제가 아이돌이나 댄서라는 직업을 하다가 잘 안되면 사업 쪽으로 가도 될까요?', 'utterance_idx': 1, 'utterance_delaytime': 0.0}, {'speaker_idx': 'T-0026', 'utterance': '물론 안되는 것은 아닙니다.', 'utterance_idx': 2, 'utterance_delaytime': 8.7}, {'speaker_idx': 'T-0026', 'utterance': '하지만 아이돌과 댄서를 하기 위해 다른 진로를 준비할 시간에 많은 투자를 했기 때문에 조금 돌아갈 수도 있습니다.', 'utterance_idx': 3, 'utterance_delaytime': 71.07}, {'speaker_idx': 'T-0026', 'utterance': '하지만 꼭 경영과를 나와야지 경영을 하는 것은 아닙니다', 'utterance_idx': 4, 'utterance_delaytime': 15.15}, {'speaker_idx': 'T-0026', 'utterance': '하지만 직접 아이돌과 댄서 생활을 해서 그쪽 일을 잘 알고 있으니 더 접근성이 뛰어날 수도 있습니다.', 'utterance_idx': 5, 'utterance_delaytime': 88.44}, {'speaker_idx': 'T-0

#### 함수 정의 + 실행

In [ ]:
def get_conversation(data):
  conversation = []
  for i in range(len(data)): # 각 학생별로
    student = data[str(i)]['conversation'] # 각 학생의 대화
    # student['conversation'] # 대화 내용
    # print(len(student))

    for j in range(len(student)):
      talk = student[j]['utterances'] # 각 학생의 진짜 대화
      utter = []

      for k in range(len(talk)):
        if k == 0: # 각 학생의 첫 번째 대화
          speaker = talk[k]['speaker_idx']
          utter = [talk[k]['utterance']]

        else: # 그 다음 대화
          if talk[k]['speaker_idx'] == speaker: # 같은 사람이면
            utter.append(talk[k]['utterance']) # 이번 발화를 저번 발화에 이어 붙이고

          else: # 다른 사람이 되면
            conversation.append({
                'speaker' : speaker,
                'utterance' : ' '.join(utter)
            })
            utter = [talk[k]['utterance']]

        speaker = talk[k]['speaker_idx'] # speaker 갱신

      if utter:
        conversation.append({
            'speaker' : speaker,
            'utterance' : ' '.join(utter)
        })

  return conversation

In [ ]:
conversation_1 = get_conversation(data1)
conversation_2 = get_conversation(data2)
conversation_3 = get_conversation(data3)

In [ ]:
len(conversation_1), len(conversation_2), len(conversation_3)

(127013, 218730, 128623)

In [ ]:
conversation = conversation_1 + conversation_2 + conversation_3
len(conversation)

474366

#### 기호, [이모티콘] 처리

** 숫자는 어떻게 처리하지

In [ ]:
def remove_punc(data):
  for sent in data:
    # '[이모티콘]' -> ~
    sent['utterance'] = sent['utterance'].replace('[이모티콘]','')
    # 링크 제거
    sent['utterance'] = re.sub(r'https://\S+', '', sent['utterance'])
    sent['utterance'] = re.sub(r'http://\S+', '', sent['utterance'])
    # , . ? ! 는 그대로 살려 (앞뒤 공백만 추가)
    sent['utterance'] = re.sub(r'([,.~!?])', r' \1 ', sent['utterance'])
    # 한글, 영어, 숫자, 앞서 살리기로 한 기호가 아니면 제거
    sent['utterance'] = re.sub(r'[^ㄱ-ㅎ가-힣,.~!?\d\w ]', '', sent['utterance'])

  return data

In [ ]:
conversation_wo_punc = remove_punc(conversation)

#### 데이터 질문 / 답변 페어링

In [ ]:
conversation_wo_punc[:5]

[{'speaker': 'S-0155', 'utterance': '안녕하세요 ~ '},
 {'speaker': 'T-0015', 'utterance': '어서 오세요  방학인데 휴가는 다녀왔어요 ? '},
 {'speaker': 'S-0155', 'utterance': '네 ! '},
 {'speaker': 'T-0015', 'utterance': '어디로 다녀왔나요 ? '},
 {'speaker': 'S-0155', 'utterance': '충주요'}]

In [ ]:
def make_pairs(data):
  pairs = pd.DataFrame()
  questions = []
  answers = []

  for i in range(len(data)-1): # 각 줄 별로

    question = data[i]['utterance']
    answer = data[i+1]['utterance']

    questions.append(question)
    answers.append(answer)


  pairs['Question'] = questions
  pairs['Answer'] = answers

  return pairs

In [ ]:
pairs = make_pairs(conversation_wo_punc)

In [ ]:
len(pairs) # 총 47만개의 pairs

474365

In [ ]:
pairs.head()

,Question,Answer
0,안녕하세요 ~,어서 오세요 방학인데 휴가는 다녀왔어요 ?
1,어서 오세요 방학인데 휴가는 다녀왔어요 ?,네 !
2,네 !,어디로 다녀왔나요 ?
3,어디로 다녀왔나요 ?,충주요
4,충주요,네 ~ 조부모님 댁인가요 ?


### Validation 데이터 가공

### 토크나이즈, 인코딩

In [ ]:
questions = pairs['Question'].tolist()
answers = pairs['Answer'].tolist()

In [ ]:
questions[-5:]

['네',
 '워크넷에서 찾아보면 그 직업의 장점 단점에 대해서도 알 수 있으니 자신의 성격과 잘 맞는지도 비교해보면 좋을 것같아요',
 '추가 질문 없으면 오늘 상담 종료할까요 ? ',
 '네 ! ',
 '선생님과 구체적으로 상담 잘 받아보시고 잘 준비해서 목표한 대학에 입학 성공하기를 응원할께요 이방에서 학생이 퇴장하면 상담은 자동 종료됩니다']

In [ ]:
# 토크나이저 정의 - 일단은 Subword Text Encoder 사용
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions + answers, target_vocab_size = 2**13)

In [ ]:
# 시작, 끝 토큰을 맨 끝에 추가해주기
start_token, end_token = tokenizer.vocab_size, tokenizer.vocab_size + 1
vocab_size = tokenizer.vocab_size + 2

In [ ]:
# 잘 됐는지 확인
print(f"두 번째 문장 : {questions[1]}")
print(f"정수 인코딩하면 : {tokenizer.encode(questions[1])}")

NameError: name 'questions' is not defined

In [ ]:
for ts in tokenizer.encode(questions[1]):
  print(f"{ts} -> {tokenizer.decode([ts])}")

702 -> 어서 
4900 -> 오세요
39 ->   
2568 -> 방학
463 -> 인데 
2654 -> 휴
210 -> 가는 
1619 -> 다녀
3376 -> 왔어요
1 ->  ? 


.....? 너 왜 휴랑 가를 나누니....?

In [ ]:
# 평균 문장 길이 확인 - 34
lens = []
for sent in questions:
  lens.append(len(sent))

sum(lens) / len(questions)

34.40457032032296

In [ ]:
# 문장 최대 길이 정하기 - 일단 40
max_length = 40

In [ ]:
def tokenize_and_pad(input, output):
  tokenized_input, tokenized_output = [], []

  for (sent1, sent2) in zip(input, output):
    tokenized1 = start_token + tokenizer.encode(sent1) + end_token
    tokenized2 = start_token + tokenizer.encode(sent2) + end_token

    tokenized_input.append(tokenized1)
    tokenized_output.append(tokenized2)

  padded_input = tf.keras.preprocessing.sequence.pad_sequences(tokenized_input, maxlen = max_length, padding = 'post')
  padded_output = tf.keras.preprocessing.sequence.pad_sequences(tokenized_output, maxlen = max_length, padding = 'post')

  return padded_input, padded_output

In [ ]:
encoded_questions, encoded_answers = tokenize_and_pad(questions, answers)

In [ ]:
# 맨 앞에 start, 맨 뒤에 end 들어감
encoded_questions[1]

array([8146,  702, 4900,   39, 2568,  463, 2654,  210, 1619, 3376,    1,
       8147,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
# 토크나이저, 인코딩 결과 저장
# /content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model
np.save('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model/encoded_train.npy', encoded_questions)
np.save('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model/encoded_test.npy', encoded_answers)
tokenizer.save_to_file('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model/tokenizer')

# 가져올 땐
# np.load( )
# SubwordTextEncoder.load_from_file( )

### 저장된 데이터 불러오기

In [ ]:
encoded_questions = np.load('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model/encoded_train.npy')
encoded_answers = np.load('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model/encoded_test.npy')
tokenizer = tfds.deprecated.text.SubwordTextEncoder.load_from_file('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model/tokenizer')

In [ ]:
encoded_questions.shape, encoded_answers.shape

((474365, 40), (474365, 40))

In [ ]:
max_length = 40

### Dataset 만들기

In [ ]:
class Dataset(Dataset):

    def __init__(self):

        self.questions = torch.LongTensor(np.load('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model/encoded_train.npy'))
        self.answers = torch.LongTensor(np.load('/content/drive/MyDrive/2. KOREA UNIV./2024 Fall/청소년 데이터 공모전/Model/encoded_test.npy'))
        self.dataset_size = len(self.questions)

    def __getitem__(self, i):

        return self.questions[i], self.answers[i]

    def __len__(self):
        return self.dataset_size

In [ ]:
train_loader = torch.utils.data.DataLoader(Dataset(),
                                           batch_size = 512, ### 64에서 수정
                                           shuffle=True,
                                           pin_memory=True)

### 구조 만들기

#### Masks

In [ ]:
def create_masks(question, reply_input, reply_target):

    def subsequent_mask(size):
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        return mask.unsqueeze(0)

    question_mask = question!=0
    question_mask = question_mask.to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1)

    reply_input_mask = reply_input!=0
    reply_input_mask = reply_input_mask.unsqueeze(1)
    reply_input_mask = reply_input_mask & subsequent_mask(reply_input.size(-1)).type_as(reply_input_mask.data)
    reply_input_mask = reply_input_mask.unsqueeze(1)
    reply_target_mask = reply_target!=0

    return question_mask, reply_input_mask, reply_target_mask

#### Positional Encoding

In [ ]:
class Embeddings(nn.Module):
  def __init__(self, vocab_size, d_model, max_len = max_length):
    super(Embeddings, self).__init__()
    self.d_model = d_model
    self.embed = nn.Embedding(vocab_size, d_model)
    self.pe = self.create_positinal_encoding(max_len, self.d_model)
    self.dropout = nn.Dropout(0.1)

  def create_positinal_encoding(self, max_len, d_model):
    pe = torch.zeros(max_len, d_model).to(device)
    for pos in range(max_len):
      for i in range(0, d_model, 2):
        pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
        pe[pos, i + 1] = math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))

    pe = pe.unsqueeze(0)
    pe = pe.to(device)
    return pe

  def forward(self, encoded_input):
    embedding = self.embed(encoded_input) * math.sqrt(self.d_model)
    embedding += self.pe[:, :embedding.size(1)]
    embedding = self.dropout(embedding)
    return embedding

#### Multi-Head Attention

In [ ]:
class MultiHeadAttention(nn.Module):

    def __init__(self, heads, d_model):

        super(MultiHeadAttention, self).__init__()
        assert d_model % heads == 0

        self.d_k = d_model // heads

        self.heads = heads
        self.dropout = nn.Dropout(0.1)
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.concat = nn.Linear(d_model, d_model)

    def forward(self, query, key, value, mask):

        query = self.query(query)
        key = self.key(key)
        value = self.value(value)

        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        scores = torch.matmul(query, key.permute(0,1,3,2)) / math.sqrt(self.d_k)

        scores = scores.masked_fill(mask == 0, -1e9)
        weights = F.softmax(scores, dim = -1)
        weights = self.dropout(weights)
        context = torch.matmul(weights, value)

        context = context.permute(0,2,1,3).contiguous().view(context.shape[0], -1, self.heads * self.d_k)
        interacted = self.concat(context)
        return interacted

#### Feed Forward

In [ ]:
class FeedForward(nn.Module):

    def __init__(self, d_model, middle_dim = 2048):
        super(FeedForward, self).__init__()

        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):

        import torch.nn.functional as F
        out = F.relu(self.fc1(x))
        out = self.fc2(self.dropout(out)) # 0.1 (10%) 의 값들을 무작위로 생략
        return out

#### Encoder

In [ ]:
class EncoderLayer(nn.Module):

    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        return encoded

#### Decoder

In [ ]:
class DecoderLayer(nn.Module):

    def __init__(self, d_model, heads):
        super(DecoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.src_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)

    def forward(self, embeddings, encoded, src_mask, target_mask):
        query = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, target_mask)) # look-ahead mask
        query = self.layernorm(query + embeddings)
        interacted = self.dropout(self.src_multihead(query, encoded, encoded, src_mask))
        interacted = self.layernorm(interacted + query)

        feed_forward_out = self.dropout(self.feed_forward(interacted))
        decoded = self.layernorm(feed_forward_out + interacted)
        return decoded

#### Transformer 구현

In [ ]:
class Transformer(nn.Module):

    def __init__(self, d_model, heads, num_layers, tokenizer):
        super(Transformer, self).__init__()

        self.d_model = d_model
        self.vocab_size = tokenizer.vocab_size + 2 ### + 2 해줌 - 이게 맞았구나...
        self.embed = Embeddings(self.vocab_size, d_model)
        self.encoder = nn.ModuleList([EncoderLayer(d_model, heads) for _ in range(num_layers)])
        self.decoder = nn.ModuleList([DecoderLayer(d_model, heads) for _ in range(num_layers)])
        self.logit = nn.Linear(d_model, self.vocab_size)

    def encode(self, src_words, src_mask):
        src_embeddings = self.embed(src_words)
        for layer in self.encoder:
            src_embeddings = layer(src_embeddings, src_mask)
        return src_embeddings

    def decode(self, target_words, target_mask, src_embeddings, src_mask):
        tgt_embeddings = self.embed(target_words)
        for layer in self.decoder:
            tgt_embeddings = layer(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings

    def forward(self, src_words, src_mask, target_words, target_mask):
        encoded = self.encode(src_words, src_mask)
        decoded = self.decode(target_words, target_mask, encoded, src_mask)

        out = F.log_softmax(self.logit(decoded), dim = 2)
        return out

#### Learning Rate Scheduler, 손실함수

In [ ]:
class AdamWarmup:

    def __init__(self, model_size, warmup_steps, optimizer):

        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0

    def get_lr(self):
        return self.model_size ** (-0.5) * min(self.current_step ** (-0.5), self.current_step * self.warmup_steps ** (-1.5))

    def step(self):
        # Increment the number of steps each time we call the step function
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        # update the learning rate
        self.lr = lr
        self.optimizer.step()

In [ ]:
class LossWithLS(nn.Module):

    def __init__(self, size, smooth):
        super(LossWithLS, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False, reduce=False)
        self.confidence = 1.0 - smooth
        self.smooth = smooth
        self.size = size

    def forward(self, prediction, target, mask):
        prediction = prediction.view(-1, prediction.size(-1))   # (batch_size * max_words, vocab_size)
        target = target.contiguous().view(-1)   # (batch_size * max_words)
        mask = mask.float()
        mask = mask.view(-1)       # (batch_size * max_words)
        labels = prediction.data.clone()
        labels.fill_(self.smooth / (self.size - 1))
        labels.scatter_(1, target.data.unsqueeze(1), self.confidence)
        loss = self.criterion(prediction, labels)    # (batch_size * max_words, vocab_size)
        loss = (loss.sum(1) * mask).sum() / mask.sum()
        return loss

#### 학습 진행

In [ ]:
d_model = 512
heads = 8
num_layers = 4
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
epochs = 1 ########### 일단 1번만

transformer = Transformer(d_model = d_model, heads = heads, num_layers = num_layers, tokenizer = tokenizer)
transformer = transformer.to(device)
adam_optimizer = torch.optim.Adam(transformer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
transformer_optimizer = AdamWarmup(model_size = d_model, warmup_steps = 4000, optimizer = adam_optimizer)
criterion = LossWithLS(tokenizer.vocab_size, 0.1)

In [ ]:
def train(train_loader, transformer, criterion, epoch):
  transformer.train()

  # 초기값 지정
  sum_loss = 0
  count = 0

  for i, (question, answer) in enumerate(train_loader):
    samples = question.shape[0]

    question = question.to(device)
    answer = answer.to(device)

    answer_input = answer[:, :-1]
    answer_target = answer[:, 1:]

    question_mask, answer_input_mask, answer_target_mask = create_masks(question, answer_input, answer_target)

    out = transformer(question, question_mask, answer_input, answer_input_mask)

    loss = criterion(out, answer_target, answer_target_mask)

    transformer_optimizer.optimizer.zero_grad()
    loss.backward()
    transformer_optimizer.step()

    sum_loss += loss.item() * samples
    count += samples

    if i % 100 == 0:
      print(f"Epoch {epoch} - {i}/{len(train_loader)} \t Loss : {sum_loss/count}")

In [ ]:
for epoch in range(epochs):

    train(train_loader, transformer, criterion, epoch)

    state = {'epoch': epoch, 'transformer': transformer, 'transformer_optimizer': transformer_optimizer}
    torch.save(state, 'checkpoint_' + str(epoch) + '.pth.tar')

Epoch 0 - 0/927 	 Loss : 7.938730239868164
Epoch 0 - 100/927 	 Loss : 7.3365394006861315
Epoch 0 - 200/927 	 Loss : 6.917466393750699
Epoch 0 - 300/927 	 Loss : 6.641302127774768
Epoch 0 - 400/927 	 Loss : 6.441172998147713
Epoch 0 - 500/927 	 Loss : 6.277922343827055
Epoch 0 - 600/927 	 Loss : 6.131917685319897
Epoch 0 - 700/927 	 Loss : 6.000881714759641
Epoch 0 - 800/927 	 Loss : 5.881568496146898
Epoch 0 - 900/927 	 Loss : 5.774148083685771


### 출력 문장 생성

In [ ]:
def getting_answers(transformer, question, question_mask, max_len, tokenizer):

  # 평가 모드로 전환
  transformer.eval()
  start_token_index = tokenizer.vocab_size
  end_token_index = tokenizer.vocab_size + 1
  encoded = transformer.encode(question, question_mask)
  words = torch.LongTensor([[start_token_index]]).to(device) # 시작한다고 말하고

  for step in range(max_len - 1): # 문장 생성하는 동안..
    size = words.shape[1]
    target_mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
    target_mask = target_mask.to(device).unsqueeze(0).unsqueeze(0)
    decoded = transformer.decode(words, target_mask, encoded, question_mask)
    predictions = transformer.logit(decoded[:, -1])
    _, next_word = torch.max(predictions, dim = 1) # 가장 확률이 높은 다음 단어 고르기
    next_word = next_word.item()

    if next_word == end_token_index: # 문장을 끝낼 때가 되면...
      break
    words = torch.cat([words, torch.LongTensor([[next_word]]).to(device)], dim = 1) # words에 추가

  if words.dim() == 2:
    words = words.squeeze(0)
    words = words.tolist()

  # 시작 토큰 없애기
  sen_idx = [w for w in words if w != start_token_index]
  sentence = tokenizer.decode(sen_idx)

  return sentence

In [ ]:
# 주의 : 얘네 첫 번째 학습된 거를 0번째로 쳐...
checkpoint = torch.load('/content/checkpoint_0.pth.tar')
transformer = checkpoint['transformer']

<ipython-input-24-d70df40f0660>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/content/checkpoint_0.pth.tar')


### 해봅시다!

In [ ]:
while(1):
  question = input('Question: ') # 입력 문장
  if question == '그만할래':
    break

  max_len = input("Maximum Reply Length: ")
  encoded_q = tokenizer.encode(question) # 입력 문장을 encode
  question = torch.LongTensor(encoded_q).to(device).unsqueeze(0) # encode된 문장을 tensor로 변환
  question_mask = (question != 0).to(device).unsqueeze(1).unsqueeze(1)
  sentence = getting_answers(transformer, question, question_mask, int(max_len), tokenizer)
  print(sentence)

Question: 나는 요리하는 게 좋아
Maximum Reply Length: 20
네
Question: 나는 뭐를 하면 좋을까?
Maximum Reply Length: 20
네
Question: 너는 뭐를 하고 싶어?
Maximum Reply Length: 20
네
Question: 그만할래


-------